[source](../../api/alibi_detect.cd.fet.rst)

# Fisher's Exact Test

## Overview

The FET drift detector is a non-parametric drift detector. It applies [Fisher's Exact Test](https://en.wikipedia.org/wiki/Fisher%27s_exact_test) (FET) to each feature, and is intended for application to [Bernoulli distributions](https://en.wikipedia.org/wiki/Bernoulli_distribution), with binary univariate data consisting of either `(True, False)` or `(0, 1)`. This detector is ideal for use in a supervised setting, monitoring drift in a model's instance level accuracy (i.e. correct prediction = 0, and incorrect prediction = 1). 

The detector is primarily intended for univariate data, but can also be used in a multivariate setting. For multivariate data, the obtained p-values for each feature are aggregated either via the [Bonferroni](https://mathworld.wolfram.com/BonferroniCorrection.html) or the [False Discovery Rate](http://www.math.tau.ac.il/~ybenja/MyPapers/benjamini_hochberg1995.pdf) (FDR) correction. The Bonferroni correction is more conservative and controls for the probability of at least one false positive. The FDR correction on the other hand allows for an expected fraction of false positives to occur. As with other univariate detectors such as the [Kolmogorov-Smirnov](ksdrift.ipynb) detector, for high-dimensional data, we typically want to reduce the dimensionality before computing the feature-wise univariate FET tests and aggregating those via the chosen correction method. See [Dimension Reduction](https://docs.seldon.io/projects/alibi-detect/en/latest/cd/background.html#dimension-reduction) for more guidance on this.

For the $j^{th}$ feature, the FET detector considers the 2x2 contingency table between the reference data $x_j^{ref}$ and test data $x_j$ for that feature:

<center>

|             | True (1)    | False (0)   |
|:----------- |:-----------:|:-----------:|
| $x_j$       | $N_1$       | $N_0$       |
| $x_j^{ref}$ | $N^{ref}_1$ | $N^{ref}_0$ |

</center>


where $N^{ref}_1$ represents the number of 1's in the reference data (for the $j^{th}$ feature), $N^{ref}_0$ the number of 0's, and so on. These values can be used to define an odds ratio:

$$
\widehat{OR} = \frac{\frac{N_1}{N_0}}{\frac{N^{ref}_1}{N^{ref}_0}}
$$

The null hypothesis is $H_0: \widehat{OR}=1$. In other words, the proportion of 1's to 0's is unchanged between the test and reference distributions, such that the odds of 1's vs 0's is independent of whether the data is drawn from the reference or test distribution. The offline FET detector can perform one-sided or two-sided tests, with the alternative hypothesis set by the `alternative` keyword argument:

- If `alternative='greater'`, the alternative hypothesis is $H_a: \widehat{OR}>1$  i.e. proportion of 1's versus 0's has increased compared to reference distribution.
- If `alternative='less'`, the alternative hypothesis is $H_a: \widehat{OR}<1$ i.e. the proportion of 1's versus 0's has decreased compared to the reference distribution.
- If `alternative='two-sided'`, the alternative hypothesis is $H_a: \widehat{OR} \ne 1$ i.e. the proportion of 1's versus 0's has changed compared to the reference distribution.

The p-value returned by the detector is then the probability of obtaining an odds ratio at least as extreme as that observed (in the direction specified by `alternative`), assuming the null hypothesis is true. 

## Usage

### Initialize

Arguments:

* `x_ref`: Data used as reference distribution.

Keyword arguments:

* `p_val`: p-value used for significance of the FET test. If the FDR correction method is used, this corresponds to the acceptable q-value.

* `preprocess_x_ref`: Whether to already apply the (optional) preprocessing step to the reference data at initialization and store the preprocessed data. Dependent on the preprocessing step, this can reduce the computation time for the predict step significantly, especially when the reference dataset is large. Defaults to *True*. It is possible that it needs to be set to *False* if the preprocessing step requires statistics from both the reference and test data, such as the mean or standard deviation.

* `update_x_ref`: Reference data can optionally be updated to the last N instances seen by the detector or via [reservoir sampling](https://en.wikipedia.org/wiki/Reservoir_sampling) with size N. For the former, the parameter equals *{'last': N}* while for reservoir sampling *{'reservoir_sampling': N}* is passed.

* `preprocess_fn`: Function to preprocess the data before computing the data drift metrics. Typically a dimensionality reduction technique.

* `correction`: Correction type for multivariate data. Either *'bonferroni'* or *'fdr'* (False Discovery Rate).

* `alternative`: Defines the alternative hypothesis. Options are *'greater'* (default), *'less'* or *'two-sided'*.

* `n_features`: Number of features used in the FET test. No need to pass it if no preprocessing takes place. In case of a preprocessing step, this can also be inferred automatically but could be more expensive to compute.

* `input_shape`: Shape of input data.

* `data_type`: can specify data type added to metadata. E.g. *'tabular'* or *'image'*.

Initialized drift detector example:

```python
from alibi_detect.cd import FETDrift

cd = FETDrift(x_ref, p_val=0.05)
```

### Detect Drift

We detect data drift by simply calling `predict` on a batch of instances `x`. We can return the feature-wise p-values before the multivariate correction by setting `return_p_val` to *True*. The drift can also be detected at the feature level by setting `drift_type` to *'feature'*. No multivariate correction will take place since we return the output of *n_features* univariate tests. For drift detection on all the features combined with the correction, use *'batch'*. `return_p_val` equal to *True* will also return the threshold used by the detector (either for the univariate case or after the multivariate correction).

The prediction takes the form of a dictionary with `meta` and `data` keys. `meta` contains the detector's metadata while `data` is also a dictionary which contains the actual predictions stored in the following keys:

* `is_drift`: 1 if the sample tested has drifted from the reference data and 0 otherwise.

* `p_val`: contains feature-level p-values if `return_p_val` equals *True*.

* `threshold`: for feature-level drift detection the threshold equals the p-value used for the significance of the FET test. Otherwise the threshold after the multivariate correction (either *bonferroni* or *fdr*) is returned.

* `distance`: Feature-wise test statistics between the reference data and the new batch if `return_distance` equals *True*. In this case, the test statistics correspond to the odds ratios. 


```python
preds = cd.predict(x, drift_type='batch', return_p_val=True)
```

### Saving and loading

Coming soon!

## Examples

[Supervised drift detection on the penguins dataset](../../examples/cd_supervised_penguins.ipynb)